# Regresion Lineal por el metodo de Ordinary Least Squares

Los modelos predictivos de Scikit-learn se definen como clases que tienen al menos cuatro métodos:
* ``__init__``: el constructor, recibe como argumentos los meta-parámetros del modelo y los almacena
* ``fit``: recibe como argumentos X e y, entrena el modelo (calcula sus parámetros) y lo devuelve.
* ``predict``: recibe como argumento X, calcula las predicciones y las devuelve.
* ``score``: recibe como argumentos X e y, realiza las predicciones sobre X y calcula con ellas el error de aproximación a y. Por defecto aplica la métrica ``r2_score`` a los modelos de regresión y la métrica ``accuracy`` a los de clasifiación.

Programa a continuación los métodos fit y predict de la regresión lineal mediante el método de Ordinary Least Squares.

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import r2_score

class OLSLinearRegression(BaseEstimator, RegressorMixin):

    def __init__(self):
        pass

    def fit(self, X, y):
        # YOUR CODE HERE
        raise NotImplementedError()
        return self

    def predict(self, X):
        # YOUR CODE HERE
        raise NotImplementedError()

    def score(self, X, y):
        preds = self.predict(X)
        return r2_score(y, preds)

La regresión lineal con el método de OLS ya está disponible en Scikit-learn mediante la clase ``LinearRegression`` del submódulo ``linear_model``. Si has programado el método de OLS correctamente, debería producir los mismos resultados que ``LinearRegression`` sobre los datasets de regresión de Boston Housing y Diabetes, que se obtienen con las funciones de Scikit-learn ``load_boston`` y ``load_diabetes``, respectivamente:

In [ ]:
from sklearn.datasets import load_boston, load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

for loader in (load_boston, load_diabetes):
    X, y = loader(return_X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

    ols_lr = OLSLinearRegression()
    lr = LinearRegression()

    ols_lr.fit(X_train, y_train)
    lr.fit(X_train, y_train)

    ols_lr_preds = ols_lr.predict(X_test)
    lr_preds = lr.predict(X_test)
    np.testing.assert_array_almost_equal(ols_lr_preds, lr_preds, decimal=3, err_msg='Las predicciones difieren demasiado')

    ols_lr_score = ols_lr.score(X_test, y_test)
    lr_score = lr.score(X_test, y_test)
    print(loader.__name__ + ':\n\tR2 OLS: ' + str(ols_lr_score) + '\n\tR2 LS: ' + str(lr_score))
    np.testing.assert_almost_equal(ols_lr_score, lr_score, decimal=2, err_msg='Los scores difieren demasiado')

# Regresion Lineal por el metodo de Stochastic Gradient Descent

En esta ocasión debes implementar los métodos ``fit`` y ``predict`` de la regresión lineal mediante el método de descenso por gradiente estocástico:

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import r2_score


class SGDLinearRegression(BaseEstimator, RegressorMixin):

    def __init__(self, max_iter=1000, eta=0.01):
        self.max_iter = max_iter
        self.eta = eta

    def fit(self, X, y):
        # YOUR CODE HERE
        raise NotImplementedError()
        return self

    def predict(self, X):
        # YOUR CODE HERE
        raise NotImplementedError()

    def score(self, X, y):
        preds = self.predict(X)
        return r2_score(y, preds)

Se puede comprobar si los resultados son similares a los obtenidos con el modelo ``SGDRegressor`` de Scikit-learn. En este caso hay que tomar la precaución de estandarizar los datos de entrada, ya que los métodos basados en descenso por gradiente son sensibles a la escala de las variables:

In [ ]:
from sklearn.datasets import load_boston, load_diabetes
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

for loader in (load_boston, load_diabetes):
    X, y = loader(return_X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

    sgd_lr = Pipeline([('stds', StandardScaler()), ('sgd_lr', SGDLinearRegression())])
    sgd = Pipeline([('stds', StandardScaler()), ('sgd', SGDRegressor(penalty=None, shuffle=False, learning_rate='constant'))])

    sgd_lr.fit(X_train, y_train)
    sgd.fit(X_train, y_train)

    sgd_lr_score = sgd_lr.score(X_test, y_test)
    sgd_score = sgd.score(X_test, y_test)
    print(loader.__name__ + ':\n\tR2 SGDLR: ' + str(sgd_lr_score) + '\n\tR2 SGDR: ' + str(sgd_score))
    np.testing.assert_almost_equal(sgd_lr_score, sgd_score, decimal=1, err_msg='Los scores difieren demasiado')

# Boston Housing con redes neuronales

Una vez entendidos los modelos lineales, estamos en condiciones de probar otros modelos más expresivos como las redes neuronales. Vamos a comprobar su efectividad con un conjunto de datos clásico en Aprendizaje Automático: Boston Housing.

El dataset está disponible en la página web de UCI (https://archive.ics.uci.edu/ml/datasets/Housing), donde, aparte de descargarlo, podéis ver el tipo de variables que contiene, que es un dataset de regresión, y algunos detalles más. Por suerte para nosotros, scikit-learn ya dispone de una función que carga los datos de este dataset en nuestro espacio de trabajo:

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()
print(boston.data.shape)
print(str(boston.data[0]))
print(str(boston.target[0]))

Para poder evaluar de forma fiable los modelos, lo primero que deberíamos hacer es definir folds para cross-validation o separar una parte de los datos como conjunto de test, por ejemplo un 10% de la muestra:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size=0.1)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

A continuación, vamos a crear el modelo base contra el que comparar nuestras redes neuronales. Dado que el problema es de regresión, lo más básico que podemos utilizar es una regresión lineal. Usaremos la ya vista anteriormente, que se entrena mediante descenso por gradiente. Estos modelos, así como los de redes neuronales, necesitan una estandarización previa de los datos, por lo que vamos a construir un pipeline de procesamiento en el que la primera componente será un estandarizador y la segunda el modelo predictivo:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
estimator = Pipeline([('standardizer', StandardScaler()), ('predictor', SGDRegressor())])

Ahora podemos entrenar nuestro modelo con el conjunto de datos de entrenamiento y evaluarlo con el conjunto de datos de test:

In [ ]:
estimator.fit(X_train, y_train)
print(estimator.score(X_test, y_test))

El resultado es mejorable, pero ya tenemos nuestro modelo base contra el que comparar. ¿Serías capaz de programar el mismo pipeline utilizando un perceptrón en lugar de una regresión lineal? La clase de scikit-learn que debes usar es sklearn.neural_network.MLPRegressor. Ten en cuenta que el número y tamaño de las capas ocultas se especifica con el meta-parámetro hidden_layer_sizes (por ejemplo, hidden_layer_sizes=() para un perceptrón simple y hidden_layer_sizes=(100, 100) para un perceptrón con dos capas ocultas de tamaño 100). Es probable que el modelo necesite muchas iteraciones de entrenamiento para converger. Puedes indicar el número máximo de iteraciones con el meta-parámetro max_iter (por ejemplo, max_iter=1000).

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Hay modelos que pueden mejorarse a través de términos de regularización. La regresión lineal se puede regularizar con un término L2, lo que nos da el modelo de regresión ridge. Las redes neuronales son los modelos más flexibles en este sentido. Vamos a ver cómo funciona un modelo ridge. Debemos especificar el coeficiente de regularización en el meta-parámetro alpha. Los valores razonables suelen estar entre 0 y 1:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
estimator = Pipeline([('standardizer', StandardScaler()), ('predictor', Ridge(alpha=0.5))])
estimator.fit(X_train, y_train)
print(estimator.score(X_test, y_test))

El modelo MLPRegressor también tiene un parámetro alpha. ¿Puedes probar el modelo añadiendo regularización?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Si has elegido bien el valor del coeficiente alpha, habrás visto una ligera mejora en tu modelo. Normalmente la búsqueda de meta-parámetros se automatiza con métodos como la búsqueda en rejilla o la búsqueda aleatoria. Vamos a probar la búsqueda en rejilla con el modelo ridge para su parámetro alpha:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
estimator = Pipeline([('standardizer', StandardScaler()), ('predictor', Ridge())])
search = GridSearchCV(estimator, {'predictor__alpha': [0.0, 0.25, 0.5, 0.75, 1.0]})
search.fit(X_train, y_train)
print(search.best_params_)
print(search.score(X_test, y_test))

¿Puedes hacer lo mismo con el parámetro alpha del perceptrón multicapa? Puedes probar más valores aparte de 0, 0.25, 0.5, 0.75 y 1.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# Digits con redes neuronales

Ahora vamos a experimentar con las redes neuronales en el problema Digits.

La información sobre el dataset se puede encontrar en http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html. Contiene imágenes de 8x8 píxeles de dígitos manuscritos, y el objetivo es clasificarlos correctamente dentro de las categorías de 0 a 9. Podemos descargarlo directamente utilizando scikit-learn:

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
print(digits.data.shape)
print(str(digits.data[0]))
print(str(digits.target[0]))

Podemos ver qué tipo de imágenes hay en el dataset:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
num_images = 10
for index, image in enumerate(digits.images[:num_images]):
    plt.subplot(2, num_images, index + num_images + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
plt.show()

Como siempre, lo correcto es hacer folds para cross-validation o al menos separar algunos patrones para test:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.1)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

Vamos a probar qué tal resuelve este problema de clasificación un perceptrón simple con alpha=0.1 (http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html):

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
estimator = Pipeline([('standardizer', StandardScaler()), ('predictor', MLPClassifier(hidden_layer_sizes=(), alpha=0.1, max_iter=10000))])
estimator.fit(X_train, y_train)
print(estimator.score(X_test, y_test))

Scikit-learn está bastante limitado a la hora de experimentar con redes neuronales, ya que no permite crear redes de tipo convolucional o recurrente. Este problema tiene claramente una estructura espacial (los píxeles de cada imagen están colocados de una determinada forma, y su orden es importante), por lo que probablemente una red convolucional sea capaz de mejorar los resultados. Para probar este tipo de modelos, podemos utilizar la librería Keras (https://keras.io/):

In [ ]:
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

K.set_image_data_format('channels_first')

input_shape = (1, 8, 8)
X_train_keras = X_train.reshape(X_train.shape[0], *input_shape)
X_test_keras = X_test.reshape(X_test.shape[0], *input_shape)
X_train_keras = X_train_keras.astype('float32')
X_test_keras = X_test_keras.astype('float32')
X_train_keras /= 16
X_test_keras /= 16
num_classes = 10
y_train_keras = to_categorical(y_train, num_classes)
y_test_keras = to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(1, 8, 8)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
model.fit(X_train_keras, y_train_keras, batch_size=32, epochs=1000, verbose=0, validation_split=0.1)
score = model.evaluate(X_test_keras, y_test_keras, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

La red convolucional que acabamos de construir tiene únicamente una capa convolucional y una capa "normal" (fully-connected, densa). Podemos mejorar los resultados modificándola para que tenga dos capas convolucionales seguidas de dos capas densas. ¿Puedes hacerlo? Ten en cuenta que hay que controlar el tamaño de los kernels de las convoluciones (en este caso usamos (2, 2). Puedes ver documentación al respecto en http://deeplearning.net/tutorial/lenet.html.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()